In [45]:
# Import necessary packages for notebook
import json
import pandas as pd
from pyspark.sql.functions import explode, split
from pyspark.sql import SparkSession, Row
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, TimestampType
import warnings

In [50]:
spark = SparkSession \
        .builder \
        .appName("ExtractEventsJob") \
        .getOrCreate()

In [58]:
games = raw_games.select(raw_games.value.cast('string'))
extracted_games = games.rdd.map(lambda x: json.loads(x.value)).toDF()

Column<b'_1'>

In [51]:
raw_games = spark \
        .read \
        .format("kafka") \
        .option("kafka.bootstrap.servers", "kafka:29092") \
        .option("subscribe", "games") \
        .option("startingOffsets", "earliest") \
        .option("endingOffsets", "latest") \
        .load()

In [52]:
new_games = raw_games \
        .select(raw_games.value.cast('string').alias('raw'),
                raw_games.timestamp.cast('string')) \
        # .filter(is_purchase('raw'))

In [44]:
extracted_games = new_games \
        .rdd \
        .map(lambda r: json.loads(r.raw)) \
        .toDF()
extracted_games.printSchema()
extracted_games.show()

root
 |-- _1: map (nullable = true)
 |    |-- key: string
 |    |-- value: long (valueContainsNull = true)
 |-- _2: map (nullable = true)
 |    |-- key: string
 |    |-- value: long (valueContainsNull = true)
 |-- _3: map (nullable = true)
 |    |-- key: string
 |    |-- value: long (valueContainsNull = true)
 |-- _4: map (nullable = true)
 |    |-- key: string
 |    |-- value: long (valueContainsNull = true)
 |-- _5: map (nullable = true)
 |    |-- key: string
 |    |-- value: long (valueContainsNull = true)
 |-- _6: map (nullable = true)
 |    |-- key: string
 |    |-- value: long (valueContainsNull = true)
 |-- _7: map (nullable = true)
 |    |-- key: string
 |    |-- value: long (valueContainsNull = true)
 |-- _8: map (nullable = true)
 |    |-- key: string
 |    |-- value: long (valueContainsNull = true)
 |-- _9: map (nullable = true)
 |    |-- key: string
 |    |-- value: long (valueContainsNull = true)
 |-- _10: map (nullable = true)
 |    |-- key: string
 |    |-- value: long (

In [14]:
# Set variavble to pull a key and value as as string
games = raw_games.select(raw_games.value.cast('string'))
games

DataFrame[value: string]

In [31]:
a = pd.read_csv(a, sep=",")

In [36]:
a

'[{"TimeRemaining": null, "HomeScoreQuarter1": 7, "HomeScoreQuarter2": 11, "HomeScoreQuarter3": 0, "HomeScoreQuarter4": 7, "Distance": "Scrambled", "AwayTeamID": 5, "UnderPayout": -180, "OverUnder": 74.1, "Has4thQuarterStarted": true, "AwayTeam": "CAR", "ForecastTempLow": 51, "PointSpread": -2.4, "GeoLong": null, "Closed": true, "QuarterDescription": "Final", "YardLineTerritory": null, "Channel": "FOX", "Status": "Final", "Week": 8, "Down": null, "ScoreID": 17790, "Has2ndQuarterStarted": true, "GameKey": "202110802", "Has3rdQuarterStarted": true, "ForecastWindSpeed": 7, "AwayScoreOvertime": 0, "Season": 2021, "PointSpreadHomeTeamMoneyLine": -186, "SeasonType": 1, "ForecastDescription": "Clear Sky", "GlobalGameID": 17790, "NeutralVenue": false, "Has1stQuarterStarted": true, "ForecastWindChill": 51, "HomeRotationNumber": 400, "GlobalAwayTeamID": 5, "Date": "2021-10-31T13:00:00", "ForecastTempHigh": 53, "OverPayout": -169, "IsOvertime": false, "Day": "2021-10-31T00:00:00", "AwayRotationNu

In [41]:
# Adjust schema to create table for variables related to our research questions
final_schema = StructType([StructField('AwayTeam', StringType(), True),
                     StructField('HomeTeam', StringType(), True),
                     StructField('Channel', StringType(), True)])

In [64]:
extracted_games = games.rdd.map(lambda x: json.loads(x.value)).toDF()

In [ ]:
extracted_games = games.rdd.map(lambda x: json.loads(x.value)).toDF(schema = final_schema)

In [67]:
extracted_games.show()

+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|                  _1|                  _2|                  _3|                  _4|                  _5|                  _6|                  _7|                  _8|                  _9|                 _10|                 _11|                 _12|                 _13|
+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|Map(PointSpread -...|Map(PointSpread -...|Map(PointSpread -...|Map(PointSpread -...|Map(PointSpread -...|Map(PointSpread -...|Map(PointSpread -...|Map(PointSpread -...|Map(Po